In [ ]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyoff
from scipy import stats
import matplotlib as mpl
%matplotlib inline

In [ ]:
from datetime import timedelta
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer, InterclusterDistance
from yellowbrick.contrib.scatter import ScatterVisualizer
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabasz_score, adjusted_rand_score
from matplotlib.ticker import FixedLocator, FixedFormatter
from sklearn.manifold import TSNE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

customer_tab= pd.read_csv('/content/drive/MyDrive/base_donnée.csv')
segment_rfm =pd.read_csv('https://raw.githubusercontent.com/bachir151/Projet-openclassrooms/master/P4/table_segmentation.csv')

Mounted at /content/drive


In [ ]:
segment_rfm.rename(columns={'Nb_inactive_days':'recency','montant':'monetary'}, inplace=True)

In [ ]:
segment_rfm.head(3)

,customer_unique_id,recency,frequency,monetary,note_com,frais_livraison,temps_moyen_livraison,Alimentation et Boissons,Auto et Moto,Autres,Livres et Médias,Maison et Jardin,Mode,Santé et Beauté,Sports et Loisirs,Électronique
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90,5.0,12.00,6.0,0,0,0,0,1,0,0,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19,4.0,8.29,3.0,0,0,0,0,0,0,1,0,0
2,0000f46a3911fa3c0805444483337064,536,1,86.22,3.0,17.22,25.0,0,0,0,0,0,0,0,0,0


In [ ]:
customer_tab.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,latitude,longitude,order_id,order_status,order_purchase_timestamp,...,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,catégorie
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,-20.513713,-47.396644,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,...,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0,office_furniture,Autres
1,97ee81fede5757f1ab031e83d1a59f56,452fab1c62b178922fc011b0eb89c1b9,47800,barreiras,BA,-12.147031,-44.996092,a4866dfc455353f48014a2b9a86608a2,delivered,2018-01-11 16:47:03,...,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0,office_furniture,Autres
2,97ee81fede5757f1ab031e83d1a59f56,452fab1c62b178922fc011b0eb89c1b9,47800,barreiras,BA,-12.147031,-44.996092,a4866dfc455353f48014a2b9a86608a2,delivered,2018-01-11 16:47:03,...,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0,office_furniture,Autres


In [ ]:
customer_tab['order_purchase_timestamp'] = pd.to_datetime(customer_tab['order_purchase_timestamp'])

In [ ]:
date_min = customer_tab['order_purchase_timestamp'].min()
date_max = customer_tab['order_purchase_timestamp'].max()

In [ ]:
date_min

Timestamp('2016-10-03 09:44:50')

In [ ]:
date_max

Timestamp('2018-08-29 15:00:37')

In [ ]:
(date_max - date_min).days

695

In [ ]:
customer_tab.groupby('customer_unique_id')['order_purchase_timestamp'].count()

customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    1
0000b849f77a49e4a4ce2b2a4ca5be3f    1
0000f46a3911fa3c0805444483337064    1
0000f6ccb0745a6a4b88665a16c9f078    1
0004aac84e0df4da2b147fca70cf8255    1
                                   ..
fffcf5a5ff07b0908bd4e2dbc735a684    2
fffea47cd6d3cc0a88bd621562a9d061    1
ffff371b4d645b6ecea244b27531430a    1
ffff5962728ec6157033ef9805bacc48    1
ffffd2657e2aad2907e67c3e9daecbeb    1
Name: order_purchase_timestamp, Length: 92501, dtype: int64

In [ ]:
segment_rfm

,customer_unique_id,recency,frequency,monetary,note_com,frais_livraison,temps_moyen_livraison,Alimentation et Boissons,Auto et Moto,Autres,Livres et Médias,Maison et Jardin,Mode,Santé et Beauté,Sports et Loisirs,Électronique
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90,5.0,12.00,6.0,0,0,0,0,1,0,0,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19,4.0,8.29,3.0,0,0,0,0,0,0,1,0,0
2,0000f46a3911fa3c0805444483337064,536,1,86.22,3.0,17.22,25.0,0,0,0,0,0,0,0,0,0
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62,4.0,17.63,20.0,0,0,0,0,0,0,0,0,1
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89,5.0,16.89,13.0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92488,fffcf5a5ff07b0908bd4e2dbc735a684,446,2,4134.84,5.0,248.71,27.0,0,0,0,0,0,0,2,0,0
92489,fffea47cd6d3cc0a88bd621562a9d061,261,1,84.58,4.0,19.69,30.0,0,0,0,0,0,0,1,0,0
92490,ffff371b4d645b6ecea244b27531430a,567,1,112.46,5.0,22.56,14.0,0,1,0,0,0,0,0,0,0
92491,ffff5962728ec6157033ef9805bacc48,118,1,133.69,5.0,18.69,11.0,0,0,0,0,0,1,0,0,0


In [ ]:
dates = customer_tab[['customer_unique_id','order_purchase_timestamp']].copy()
dates.columns = ['customer_unique_id', 'Date_de_commande']

In [ ]:
dates

,customer_unique_id,Date_de_commande
0,861eff4711a542e4b93843c6dd7febb0,2017-05-16 15:05:35
1,452fab1c62b178922fc011b0eb89c1b9,2018-01-11 16:47:03
2,452fab1c62b178922fc011b0eb89c1b9,2018-01-11 16:47:03
3,613259d64ba972e49dc25b019b7918c3,2017-12-07 16:41:50
4,aaad33c5051196b5e73c7d65b9072e1d,2018-03-13 20:11:20
...,...,...
114552,20120882a566b99eb67c104dc05d896f,2017-07-09 15:17:43
114553,7c0b4f7719caf8b29a4c8dbd5a1446c9,2018-06-11 16:10:21
114554,640ac18151727c99757e30b92253dbc0,2018-04-29 17:40:51
114555,48f2f72862e37acc97f50b4484ee2af4,2017-10-17 21:50:00


In [ ]:
segment_rfm

,customer_unique_id,recency,frequency,monetary,note_com,frais_livraison,temps_moyen_livraison,Alimentation et Boissons,Auto et Moto,Autres,Livres et Médias,Maison et Jardin,Mode,Santé et Beauté,Sports et Loisirs,Électronique
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90,5.0,12.00,6.0,0,0,0,0,1,0,0,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19,4.0,8.29,3.0,0,0,0,0,0,0,1,0,0
2,0000f46a3911fa3c0805444483337064,536,1,86.22,3.0,17.22,25.0,0,0,0,0,0,0,0,0,0
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62,4.0,17.63,20.0,0,0,0,0,0,0,0,0,1
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89,5.0,16.89,13.0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92488,fffcf5a5ff07b0908bd4e2dbc735a684,446,2,4134.84,5.0,248.71,27.0,0,0,0,0,0,0,2,0,0
92489,fffea47cd6d3cc0a88bd621562a9d061,261,1,84.58,4.0,19.69,30.0,0,0,0,0,0,0,1,0,0
92490,ffff371b4d645b6ecea244b27531430a,567,1,112.46,5.0,22.56,14.0,0,1,0,0,0,0,0,0,0
92491,ffff5962728ec6157033ef9805bacc48,118,1,133.69,5.0,18.69,11.0,0,0,0,0,0,1,0,0,0


In [ ]:
df = segment_rfm.merge(dates, on = 'customer_unique_id', how = 'inner')

## Autre méthode 

In [ ]:
df.shape

(114549, 17)

In [ ]:
df

,customer_unique_id,recency,frequency,monetary,note_com,frais_livraison,temps_moyen_livraison,Alimentation et Boissons,Auto et Moto,Autres,Livres et Médias,Maison et Jardin,Mode,Santé et Beauté,Sports et Loisirs,Électronique,Date_de_commande
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90,5.0,12.00,6.0,0,0,0,0,1,0,0,0,0,2018-05-10 10:56:27
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19,4.0,8.29,3.0,0,0,0,0,0,0,1,0,0,2018-05-07 11:11:27
2,0000f46a3911fa3c0805444483337064,536,1,86.22,3.0,17.22,25.0,0,0,0,0,0,0,0,0,0,2017-03-10 21:05:03
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62,4.0,17.63,20.0,0,0,0,0,0,0,0,0,1,2017-10-12 20:29:41
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89,5.0,16.89,13.0,0,0,0,0,0,0,0,0,1,2017-11-14 19:45:42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114544,fffcf5a5ff07b0908bd4e2dbc735a684,446,2,4134.84,5.0,248.71,27.0,0,0,0,0,0,0,2,0,0,2017-06-08 21:00:36
114545,fffea47cd6d3cc0a88bd621562a9d061,261,1,84.58,4.0,19.69,30.0,0,0,0,0,0,0,1,0,0,2017-12-10 20:07:56
114546,ffff371b4d645b6ecea244b27531430a,567,1,112.46,5.0,22.56,14.0,0,1,0,0,0,0,0,0,0,2017-02-07 15:49:16
114547,ffff5962728ec6157033ef9805bacc48,118,1,133.69,5.0,18.69,11.0,0,0,0,0,0,1,0,0,0,2018-05-02 15:17:41


In [ ]:
date_min = df['Date_de_commande'].min()
date_max = df['Date_de_commande'].max()

In [ ]:
df['Nb_jour_du_programme'] = (df['Date_de_commande'] - date_min).dt.days 

In [ ]:
df

,customer_unique_id,recency,frequency,monetary,note_com,frais_livraison,temps_moyen_livraison,Alimentation et Boissons,Auto et Moto,Autres,Livres et Médias,Maison et Jardin,Mode,Santé et Beauté,Sports et Loisirs,Électronique,Date_de_commande,Nb_jour_du_programme
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90,5.0,12.00,6.0,0,0,0,0,1,0,0,0,0,2018-05-10 10:56:27,584
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19,4.0,8.29,3.0,0,0,0,0,0,0,1,0,0,2018-05-07 11:11:27,581
2,0000f46a3911fa3c0805444483337064,536,1,86.22,3.0,17.22,25.0,0,0,0,0,0,0,0,0,0,2017-03-10 21:05:03,158
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62,4.0,17.63,20.0,0,0,0,0,0,0,0,0,1,2017-10-12 20:29:41,374
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89,5.0,16.89,13.0,0,0,0,0,0,0,0,0,1,2017-11-14 19:45:42,407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114544,fffcf5a5ff07b0908bd4e2dbc735a684,446,2,4134.84,5.0,248.71,27.0,0,0,0,0,0,0,2,0,0,2017-06-08 21:00:36,248
114545,fffea47cd6d3cc0a88bd621562a9d061,261,1,84.58,4.0,19.69,30.0,0,0,0,0,0,0,1,0,0,2017-12-10 20:07:56,433
114546,ffff371b4d645b6ecea244b27531430a,567,1,112.46,5.0,22.56,14.0,0,1,0,0,0,0,0,0,0,2017-02-07 15:49:16,127
114547,ffff5962728ec6157033ef9805bacc48,118,1,133.69,5.0,18.69,11.0,0,0,0,0,0,1,0,0,0,2018-05-02 15:17:41,576


In [ ]:
df['group_jours'] = df['Nb_jour_du_programme'] // 15

In [ ]:
df

,customer_unique_id,recency,frequency,monetary,note_com,frais_livraison,temps_moyen_livraison,Alimentation et Boissons,Auto et Moto,Autres,Livres et Médias,Maison et Jardin,Mode,Santé et Beauté,Sports et Loisirs,Électronique,Date_de_commande,Nb_jour_du_programme,group_jours
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90,5.0,12.00,6.0,0,0,0,0,1,0,0,0,0,2018-05-10 10:56:27,584,38
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19,4.0,8.29,3.0,0,0,0,0,0,0,1,0,0,2018-05-07 11:11:27,581,38
2,0000f46a3911fa3c0805444483337064,536,1,86.22,3.0,17.22,25.0,0,0,0,0,0,0,0,0,0,2017-03-10 21:05:03,158,10
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62,4.0,17.63,20.0,0,0,0,0,0,0,0,0,1,2017-10-12 20:29:41,374,24
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89,5.0,16.89,13.0,0,0,0,0,0,0,0,0,1,2017-11-14 19:45:42,407,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114544,fffcf5a5ff07b0908bd4e2dbc735a684,446,2,4134.84,5.0,248.71,27.0,0,0,0,0,0,0,2,0,0,2017-06-08 21:00:36,248,16
114545,fffea47cd6d3cc0a88bd621562a9d061,261,1,84.58,4.0,19.69,30.0,0,0,0,0,0,0,1,0,0,2017-12-10 20:07:56,433,28
114546,ffff371b4d645b6ecea244b27531430a,567,1,112.46,5.0,22.56,14.0,0,1,0,0,0,0,0,0,0,2017-02-07 15:49:16,127,8
114547,ffff5962728ec6157033ef9805bacc48,118,1,133.69,5.0,18.69,11.0,0,0,0,0,0,1,0,0,0,2018-05-02 15:17:41,576,38


In [ ]:
new_df = df.copy()
new_df = new_df.drop(['customer_unique_id','Date_de_commande','Nb_jour_du_programme'], axis = 1)

In [ ]:
df1=new_df.groupby('group_jours').agg({
    'recency' : 'median',
    'frequency' : 'median',
    'monetary': 'median', 
    'note_com' : 'median',
    'frais_livraison' : 'median',
    'temps_moyen_livraison' : 'median',
    'group_jours' : 'count'    
})

In [ ]:
#df1.group_jours.cumsum()

On peut ainsi programmer une mise à jour de notre programme après 7 mois soit le groupe 

## Méthode retenue

Nous allons calculer le delta de temps entre la première et la dernière commande de notre dataset.

In [ ]:
#min_date  = orders["order_purchase_timestamp"].min()
#max_date= orders["order_purchase_timestamp"].max()
min_date = df['Date_de_commande'].min()
max_date= df['Date_de_commande'].max()

delta_date = (max_date - min_date).days
print("Nombre de jours entre la première et la dernière commande : {}".format(delta_date) )

Nombre de jours entre la première et la dernière commande : 695


Nous allons pouvoir calculer un indice ARI en fonction du temps pour notre clustering. Nous allons pour cela vieillir artificiellement les données d'un an afin de vérifier leur stabilité. Cela nous permettra d'afficher un graphique de progression de l'indice ARI.

In [ ]:
df

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score
import warnings
warnings.simplefilter("ignore")

features = ['recency',	'frequency',	'monetary',	'note_com',	'frais_livraison',	'temps_moyen_livraison']

# Test de ARI selon un delta prédéfini :
delta_old = 336
df_old = df[df['Date_de_commande'] <= max_date - pd.Timedelta(days=delta_old)]
X_old = df_old[features]
X_old = StandardScaler().fit_transform(X_old)
#kmeans = KMeans(n_clusters=5, init='k-means++', random_state=0)
kmeans =KMeans(n_clusters=5, n_init=10, random_state=0)
kmeans_old = kmeans.fit(X_old)


# On crée une liste vide pour stocker les valeurs de l'indice ARI :
ari_list = []
# On crée une liste vide pour stocker les dates :
days = []

for i in range(int(delta_old/2)):

    # On récupère les données de notre dataset à partir de la date i :
    df_temp = df[df['Date_de_commande'] <= (max_date - pd.Timedelta(days=delta_old) + pd.Timedelta(days=i*2))]
    days.append(i*2)
    # On normalise les données :
    X = df_temp[features]
    scale = StandardScaler()
    X_scaled = scale.fit_transform(X)

    # On effectue le clustering :
    kmeans = KMeans(n_clusters=5, init='k-means++', random_state=0)
    kmeans.fit(X_scaled)
    # On calcule une prédiction sur les données :
    labels = kmeans.predict(X_scaled)
    labels_true = kmeans_old.predict(X_scaled)
    # On calcule l'indice ARI :
    ari_list.append(adjusted_rand_score(labels_true, labels))
    print("ARI for {} days old : {}".format(i*2, ari_list[-1]))



ARI for 0 days old : 1.0
ARI for 2 days old : 0.3116868641989814
ARI for 4 days old : 0.2831177747908579
ARI for 6 days old : 0.2832313790182751
ARI for 8 days old : 0.28203440560019677
ARI for 10 days old : 0.8970609457420842
ARI for 12 days old : 0.2778716443066974
ARI for 14 days old : 0.2842585383906223
ARI for 16 days old : 0.9840191833840047
ARI for 18 days old : 0.2833194849625168
ARI for 20 days old : 0.9702688796365022
ARI for 22 days old : 0.28282777156530386
ARI for 24 days old : 0.2841171291644883
ARI for 26 days old : 0.28067110109706256
ARI for 28 days old : 0.8782995427930887
ARI for 30 days old : 0.2840609808422362
ARI for 32 days old : 0.2842540038682999
ARI for 34 days old : 0.2858071423012386
ARI for 36 days old : 0.2861354106416714
ARI for 38 days old : 0.98064056714557
ARI for 40 days old : 0.2872899272790276
ARI for 42 days old : 0.286328203280278
ARI for 44 days old : 0.2866663993058403
ARI for 46 days old : 0.28212860647150295
ARI for 48 days old : 0.28543340374

KeyboardInterrupt: ignored

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


ARI for 120 days old : 0.30341957909834866


KeyboardInterrupt: ignored

In [ ]:
# On affiche le graphique :
plt.figure(figsize=(20,10))
sns.lineplot(x=days, y=ari_list, color='red', label='ARI')
plt.xlabel('Days')
plt.ylabel('ARI')
plt.title('ARI progression')
plt.show()
